# Obtención de las imágenes provenientes de Sentinel-2

EL siguiente código emplea los paquetes *planetary_computer* y *pystac_client* para establecer la conexión con la STAC API de Planetary Computer [[1]](https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/).  El paquete *rioxarray* se emplea para la lectura y visualización de las imágenes satelitales provenientes de los satélites Sentinel-2 [[2]](https://corteva.github.io/rioxarray/html/rioxarray.html).

## Imports

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
import cv2
from datetime import timedelta
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm

import planetary_computer as pc
from pystac_client import Client
import geopy.distance as distance
import rioxarray

import json

%matplotlib inline

## Definición de parámetros

In [ ]:
# Directorios de donde se leerá y almacenará la información
DATA_DIR = Path.cwd().resolve() / "data"
SENTINEL_DATA_DIR = DATA_DIR / "sentinel"

# Máximo de días hacia atrás donde se permite descargar la imagen, respecto a la fecha de la medición in situ
TIME_BUFFER_DAYS = 15

# Tamaño del detalle de la imagen (se aplica a cada dirección cardinal)
METER_BUFFER = 200

## Metadata in situ

In [ ]:
metadata = pd.read_csv(DATA_DIR / "metadata.csv")

In [ ]:
# Se convierte la variable fecha al tipo datetime
metadata.date = pd.to_datetime(metadata.date)

## Funciones

In [ ]:
def get_bounding_box(latitude, longitude, meter_buffer=50000):
    """
    Dada una latitud, longitud y buffer en metros, devuelve una superficie rectangular alrededor de las coordenadas proporcionadas en
    cada una de las direcciones cardinales

    Devuelve una lista de [minx, miny, maxx, maxy]
    """
    distance_search = distance.distance(meters=meter_buffer)

    # calcula la lat/long basada en la distancia en tierra
    # cada orientación corresponde a una dirección cardinal (sur, oeste, norte, y este)
    min_lat = distance_search.destination((latitude, longitude), bearing=180)[0]
    min_long = distance_search.destination((latitude, longitude), bearing=270)[1]
    max_lat = distance_search.destination((latitude, longitude), bearing=0)[0]
    max_long = distance_search.destination((latitude, longitude), bearing=90)[1]

    return [min_long, min_lat, max_long, max_lat]

In [ ]:
# get our date range to search, and format correctly for query
def get_date_range(date, time_buffer_days=TIME_BUFFER_DAYS):
    """
    Obtiene el rango de fechas con las que se busca en PC a partir de la fecha de medición y el límite de días atras establecido

    Devuelve un string con el rango
    """
    datetime_format = "%Y-%m-%dT"
    range_start = pd.to_datetime(date) - timedelta(days=time_buffer_days)
    date_range = f"{range_start.strftime(datetime_format)}00:00:00Z/{pd.to_datetime(date).strftime(datetime_format)}23:59:59Z"

    return date_range

In [ ]:
def crop_sentinel_image(item, bounding_box):
    """
    Dado un STAC ítem de Sentinel y la bbox en formato tupla (minx, miny, maxx, maxy), devuelve un recorte de la imagen

    Devuelve las imágenes como un vector numpy con dimensiones [(3,Px,Py), (1,Px,Py), (1,Px,Py), (1,Px,Py), (1,Px,Py)], 
    siendo Px y Py los píxeles en cada dirección del plano
    """
    (minx, miny, maxx, maxy) = bounding_box

    # true color image (TCI). 10 m de resolución
    visual_img = (
        rioxarray.open_rasterio(pc.sign(item.assets["visual"].href))
        .rio.clip_box(
            minx=minx,
            miny=miny,
            maxx=maxx,
            maxy=maxy,
            crs="EPSG:4326",
        )
        .to_numpy()
    )

    # scene classification image (SCL). 20 m de resolución
    SCL_img = (
        rioxarray.open_rasterio(pc.sign(item.assets["SCL"].href))
        .rio.clip_box(
            minx=minx,
            miny=miny,
            maxx=maxx,
            maxy=maxy,
            crs="EPSG:4326",
        )
        .to_numpy()
    )
    # 560 nm band. 10 m de resolución
    b03_img = (
        rioxarray.open_rasterio(pc.sign(item.assets["B03"].href))
        .rio.clip_box(
            minx=minx,
            miny=miny,
            maxx=maxx,
            maxy=maxy,
            crs="EPSG:4326",
        )
        .to_numpy()
    )
    # 665 nm band. 10 m de resolución
    b04_img = (
        rioxarray.open_rasterio(pc.sign(item.assets["B04"].href))
        .rio.clip_box(
            minx=minx,
            miny=miny,
            maxx=maxx,
            maxy=maxy,
            crs="EPSG:4326",
        )
        .to_numpy()
    )
    # 704 nm band. 20 m de resolución
    b05_img = (
        rioxarray.open_rasterio(pc.sign(item.assets["B05"].href))
        .rio.clip_box(
            minx=minx,
            miny=miny,
            maxx=maxx,
            maxy=maxy,
            crs="EPSG:4326",
        )
        .to_numpy()
    )


    return [visual_img, b03_img, b04_img, b05_img, SCL_img]

In [ ]:
def select_item_list(items, date, latitude, longitude):
    """
    Devuelve una lista con los ítems válidos:
    - Debe contener la coordenada de medición
    - Se encuentra dentro del rango temporal definido

    Devuelve una estructura dataframe ordenada por la diferencia temporal con respecto a la fecha de medición (de menor a mayor),
    con los detalles de los ítems
    """
    # Obtención de los atributos de los ítems
    item_details = pd.DataFrame(
        [
            {
                "datetime": item.datetime.strftime("%Y-%m-%d"),
                "platform": item.properties["platform"],
                "min_long": item.bbox[0],
                "max_long": item.bbox[2],
                "min_lat": item.bbox[1],
                "max_lat": item.bbox[3],
                "item_obj": item,
                "cloud_cover": item.properties["eo:cloud_cover"],
            }
            for item in items
        ]
    )

    # filtrado de los puntos que contienen la localización de la medición
    item_details["contains_sample_point"] = (
        (item_details.min_lat < latitude)
        & (item_details.max_lat > latitude)
        & (item_details.min_long < longitude)
        & (item_details.max_long > longitude)
    )
    item_details = item_details[item_details["contains_sample_point"] == True]
    if len(item_details) == 0:
        return (np.nan, np.nan, np.nan)

    # Se añade la diferencia temporal como atributo al df
    item_details["time_diff"] = pd.to_datetime(date) - pd.to_datetime(
        item_details["datetime"]
    )

    # return the closest imagery by time
    return item_details.sort_values(by="time_diff", ascending=True)

In [ ]:
def check_clouds(ordered_items, bounding_box):
    """
    Comprueba que el porcentaje de píxeles nubosos o erróneos sean mayores al 25% de la superficie recortada,
    a partir de la lista de ítems y el área de búsqueda

    Devuelve el ítem que cumple el criterio de nubosidad y de menor diferencia temporal con la fecha de medición.
    """
    (minx, miny, maxx, maxy) = bounding_box
    best_item = None

    for i in range(len(ordered_items)):
        cloud_image_array = (
            rioxarray.open_rasterio(
                pc.sign(ordered_items.iloc[i].item_obj.assets["SCL"].href)
            )
            .rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
            .to_numpy()
        )
        
        ## DECODIFICACION SCL SENTINEL - 2
        # 0 - No data
        # 1 - Saturated / Defective
        # 2 - Dark Area Pixels
        # 3 - Cloud Shadows
        # 4 - Vegetation
        # 5 - Bare Soils
        # 6 - Water
        # 7 - Clouds low probability / Unclassified
        # 8 - Clouds medium probability
        # 9 - Clouds high probability
        # 10 - Cirrus
        # 11 - Snow / Ice

        if np.size(
            cloud_image_array[
                (cloud_image_array != 0)
                & (cloud_image_array != 1)
                & (cloud_image_array != 2)
                & (cloud_image_array != 3)
                & (cloud_image_array != 8)
                & (cloud_image_array != 9)
                & (cloud_image_array != 10)
            ]
        ) >= 0.25 * np.size(cloud_image_array):
            best_item = ordered_items.iloc[i]
            break

    return best_item

## Extracción de las imágenes

In [ ]:
# Establece la conexión con el STAC API
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1", modifier=pc.sign_inplace
)

In [ ]:
data_subset = metadata

La siguiente celda conviene ejecutarla más de una vez en caso de que el pull falle por la conexión con la API

In [ ]:
# save outputs in dictionaries
selected_items = {} # mantiene un registro de los items seleccionados
not_possible = [] # mantiene un registro de aquellos ítems no disponibles
errored_ids = [] # mantiene un registro de aquellos ítems que han fallado durante la descarga

for row in tqdm(data_subset.itertuples(), total=len(data_subset)):
    pass
    image_array_pth = SENTINEL_DATA_DIR / f"visual/{row.uid}.npy"
    
    # se comprueba que el item no exista ya en el directorio
    try:
        with open(image_array_pth, "rb") as f:
            continue

    except:
        try:
            ## QUERY STAC API
            search_bbox = get_bounding_box(
                row.latitude, row.longitude, meter_buffer=50000
            )
            date_range = get_date_range(row.date, time_buffer_days=15)

            # se busca en el catálogo de planetary computer
            search = catalog.search(
                collections=["sentinel-2-l2a"],
                bbox=search_bbox,
                datetime=date_range,
                query={
                    "platform": {
                        "in": [
                            "Sentinel-2A",
                            "Sentinel-2B",
                        ]
                    }
                },
            )
            items = [item for item in search.get_all_items()]

            # Se comprueba que hayan ítems y se ordenan según la diferencia de fechas. Se descartan aquellos ítems que no contienen
            # el lugar de la medición
            if len(items) == 0:
                not_possible.append(row.uid)
                pass
            else:
                ordered_items = select_item_list(
                    items, row.date, row.latitude, row.longitude
                )

            # Creación de la superficie del detalle
            feature_bbox = get_bounding_box(
                row.latitude, row.longitude, meter_buffer=200
            )

            # Comprobación nubosidad
            best_item = check_clouds(ordered_items, feature_bbox)
            if best_item is None:
                not_possible.append(row.uid)
                continue

            # Registro de los ítems seleccionados
            selected_items[row.uid] = {
                "item_object": str(best_item["item_obj"]),
                "item_platform": best_item["platform"],
                "item_date": best_item["datetime"],
                "cloud_properties": best_item["cloud_cover"],
                "time_diff": str(best_item["time_diff"]),
            }

            # Obtención de las imagenes recortadas
            [
                image_array,
                b03_image_array,
                b04_image_array,
                b05_image_array,
                scl_image_array,
            ] = crop_sentinel_image(best_item["item_obj"], feature_bbox)

            # Almacenamiento
            with open(SENTINEL_DATA_DIR / f"visual/{row.uid}.npy", "wb") as f:
                np.save(f, image_array)
            with open(SENTINEL_DATA_DIR / f"b03/{row.uid}.npy", "wb") as f:
                np.save(f, b03_image_array)
            with open(SENTINEL_DATA_DIR / f"b04/{row.uid}.npy", "wb") as f:
                np.save(f, b04_image_array)
            with open(SENTINEL_DATA_DIR / f"b05/{row.uid}.npy", "wb") as f:
                np.save(f, b05_image_array)
            with open(SENTINEL_DATA_DIR / f"scl/{row.uid}.npy", "wb") as f:
                np.save(f, scl_image_array)

        # Registro de los elementos que han dado error
        except Exception as e:
            errored_ids.append(row.uid)

In [ ]:
with open(SENTINEL_DATA_DIR / "selected_items.txt", "w") as f:
    json.dump(selected_items, f)

In [ ]:
print(f"No se puedo obtener imágenes para {len(errored_ids)} elementos debido a errores")

In [ ]:
print(f"No se puedo obtener imágenes para {len(not_possible)} elementos")

## Referencias

[1] Microsoft. Microsoft Planetary Computer.Reading Data from the STAC API: https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/

[2] Corteva Agriscience. Corteva Agriscience: https://corteva.github.io/rioxarray/html/rioxarray.html

[3] Driven Data. Tick Tick Bloom: Harmful Algal Bloom Detection Challenge: https://www.drivendata.org/competitions/143/tick-tick-bloom/page/650/
